# Max-Cut (10,000 Vertices) on Dynex n.quantum computing cloud

In [ ]:
import dynex
import networkx as nx
import matplotlib
from matplotlib import pyplot as plt
from collections import defaultdict

## Simple Max-Cut graph

In [ ]:
# Create empty graph
G = nx.Graph()

In [ ]:
# Add edges to the graph (also adds nodes)
G.add_edges_from([(1,2),(1,3),(2,4),(3,4),(3,5),(4,5)])

In [ ]:
# ------- Set up our QUBO dictionary -------

# Initialize our Q matrix
Q = defaultdict(int)

# Update Q matrix for every edge in the graph
for i, j in G.edges:
    Q[(i,i)]+= -1
    Q[(j,j)]+= -1
    Q[(i,j)]+= 2

In [ ]:
# Sample on Dynex
sampleset = dynex.sample_qubo(Q, 0.0, mainnet=False, description='Dynex SDK job', num_reads=50000, annealing_time = 200, debugging=True);
print('Result:')
print(sampleset)

In [ ]:
# Grab best result
lut = sampleset.first.sample

# Interpret best result in terms of nodes and edges
S0 = [node for node in G.nodes if not lut[node]]
S1 = [node for node in G.nodes if lut[node]]
cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

print('Maxcut result:',len(cut_edges))

# Display best result
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='r')
nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='c')
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=3)
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=3)
nx.draw_networkx_labels(G, pos)

## 10,000 Vertices

In [ ]:
import numpy as np

In [ ]:
filename = 'G70.dat' 

In [ ]:
f = open(filename, "r").readlines()

In [ ]:
b_matrix=False
bin_size = f[0].split(" ")[0]
Q = np.zeros([int(bin_size) + 1, int(bin_size) + 1])
init_bin = np.zeros([int(bin_size) + 1])
init_bin[-1] = 1
for ele in f[1:]:
    i, j, v = ele.split()
    if b_matrix == True :
        Q[int(i) - 1, int(j) - 1] += int(v)
        if (int(i) != int(j)):
            Q[int(j) - 1, int(i) - 1] += int(v)
    else :
        if (int(i) == int(j)) :
            print('No edge connected at the same Node',int(i),int(j))
        else :
            Q[int(i), int(j)] += int(v)
            Q[int(j), int(i)] += int(v)
            Q[int(i), int(i)] += -int(v)
            Q[int(j), int(j)] += -int(v)

In [ ]:
# Sample on Dynex
sampleset = dynex.sample_qubo(Q, 0.0, mainnet=True, num_reads=100000, annealing_time = 10000, alpha=0.05, beta=0.2);
print('Result:')
print(sampleset)